# [CSC8101] Big Data Analytics - 2022 Spark Coursework

## Coursework overview

### Inputs

- **NYC Taxi Trips dataset** - list of recorded taxi trips, each with several characteristics, namely: distance, number of passengers, origin zone, destination zone and trip cost (total amount charged to customer).
- **NYC Zones dataset** - list of zones wherein trips can originate/terminate.

### Tasks

1. Data cleaning
  1. Remove "0 distance" and 'no passengers' records.
  2. Remove outlier records. 
2. Add new columns
  1. Join with zones dataset
  2. Compute the unit profitability of each trip
3. Zone summarisation and ranking
  1. Summarise trip data per zone
  2. Obtain the top 10 ranks according to:
    1. The total trip volume
    2. Their average profitabilitiy
    3. The total passenger volume
4. Record the total and task-specific execution times for each dataset size and format.

### How to

###### Code structure and implementation

- You must implement your solution to each task in the provided function code skeleton.
- The task-specific functions are combined together to form the full pipeline code, executed last (do not modify this code).
- Before implementing the specified function skeleton, you should develop and test your solution on separate code cells (create and destroy cells as needed).

###### Development

- Develop an initial working solution for the 'S' dataset and only then optimise it for larger dataset sizes.
- To perform vectorised operations on a DataFrame:
  - use the API docs to look for existing vectorised functions in: https://spark.apache.org/docs/latest/api/python/reference/pyspark.sql.html#functions
  - if a customised function is required (e.g. to add a new column based on a linear combination of other columns), implement your own User Defined Function (UDF). See:  https://spark.apache.org/docs/latest/sql-ref-functions-udf-scalar.html
- Use only the `pyspark.sql` API - documentation link below - (note that searching through the docs returns results from the `pyspark.sql` API together with the `pyspark.pandas` API):
  - https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.sql.html
- Periodically download your notebook to your computer as backup and safety measure against accidental file deletion.
 
###### Execution time measurement

- Execution time is calculated and returned by the Spark Engine and shown in the output region of the cell.
- To measure the execution time of a task you must perform a `collect` or similar operation (e.g. `take`) on the returned DataFrame.

## Task 0 - Read data

The code below is ready to run. **Do not modify this code**. It does the following:

- Reads the 'zones' dataset into variable 'zone_names'
- Defines the `init_trips` function that allows you to read the 'trips' dataset (from the DBFS FileStore) given the dataset size ('S' to 'XXL') and format ('parquet' or 'delta') as function arguments
- Defines the `pipeline` function, called in Task 4 to measure the execution time of the entire data processing pipeline
- Shows you how to call the `init_trips` function and display dataset characteristics (number of rows, schema)

In [0]:
## global imports
import pyspark.sql as ps
import pyspark.sql.functions as pf
import pandas as pd

# Load zone names dataset - (much faster to read small file from git than dbfs)
zones_file_url = 'https://raw.githubusercontent.com/NewcastleComputingScience/csc8101-coursework/main/02-assignment-spark/taxi_zone_names.csv'
zone_names = spark.createDataFrame(pd.read_csv(zones_file_url))

# Function to load trips dataset by selected dataset size
def init_trips(size = 'S', data_format = "parquet", taxi_folder = "/FileStore/tables/taxi"):     
    
    files = {
        'S'  : ['2021_07'],
        'M'  : ['2021'],
        'L'  : ['2020_21'],
        'XL' : ['1_6_2019', '7_12_2019'],
        'XXL': ['1_6_2019', '7_12_2019', '2020_21']
    }
    
    # validate input dataset size
    if size not in files.keys():
        print("Invalid input dataset size. Must be one of {}".format(list(files.keys())))
        return None               
    
    if data_format == "parquet":
        filenames = list(map(lambda s: f'{taxi_folder}/parquet/tripdata_{s}.parquet', files[size]))
        trips_df = spark.read.parquet(filenames[0])
        
        for name in filenames[1:]:
            trips_df = trips_df.union(spark.read.parquet(name))
            
    elif data_format == "delta":
        filenames = f"{taxi_folder}/delta/taxi-{size}-delta/"
        trips_df = spark.read.format("delta").load(filenames)
    
    else:
        print("Invalid data format. Must be one of {}".format(['parquet', 'delta']))
        return None
        
    print(
    """
    Trips dataset loaded!
    ---
      Size: {s}
      Format: {f}
      Tables loaded: {ds}
      Number of trips (dataset rows): {tc:,}
    """.format(s = size, f = data_format, ds = filenames, tc = trips_df.count()))
    
    return trips_df

# helper function to print dataset row count
def print_count(df):
    print("Row count: {t:,}".format(t = df.count()))

def pipeline(trips_df, with_task_12 = False, zones_df = zone_names):
    # Do not edit
    #---

    ## Task 1.1
    _trips_11 = t11_remove_zeros(trips_df)

    ## Task 1.2
    if with_task_12:
        _trips_12 = t12_remove_outliers(_trips_11)
    else:
        _trips_12 = _trips_11

    ## Task 2.1
    _trips_21 = t21_join_zones(_trips_12, zones_df = zone_names)

    ## Task 2.2
    _trips_22 = t22_calc_profit(_trips_21)

    ## Task 3.1
    _graph = t31_summarise_trips(_trips_22)

    ## Task 3.2
    _zones = t32_summarise_zones_pairs(_graph)

    _top10_trips     = t32_top10_trips(_zones)
    _top10_profit    = t32_top10_profit(_zones)
    _top10_passenger = t32_top10_passenger(_zones)
    
    return([_top10_trips, _top10_profit, _top10_passenger])

In [0]:
# CHANGE the value of argument 'size' to record the pipeline execution times for increasing dataset sizes
SIZE = 'S'
DATA_FORMAT = 'parquet'


# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)

# uncomment line only for small datasets
# trips.take(1)


    Trips dataset loaded!
    ---
      Size: S
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2021_07.parquet']
      Number of trips (dataset rows): 2,898,033
    


In [0]:
print_count(trips)

Row count: 2,898,033


In [0]:
# dataset schemas
trips.printSchema()

root
 |-- index: long (nullable = true)
 |-- VendorID: double (nullable = true)
 |-- tpep_pickup_datetime: string (nullable = true)
 |-- tpep_dropoff_datetime: string (nullable = true)
 |-- passenger_count: double (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: double (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: long (nullable = true)
 |-- DOLocationID: long (nullable = true)
 |-- payment_type: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- cab_type: string (nullable = true)
 |-- lpep_pickup_datetime: string (nullable = true)
 |-- lpep_dropoff_datetime: string (nullable = true)
 |--

In [0]:
display(trips[['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount']].take(5))

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3


In [0]:
zone_names.printSchema()

root
 |-- LocationID: long (nullable = true)
 |-- Borough: string (nullable = true)
 |-- Zone: string (nullable = true)
 |-- service_zone: string (nullable = true)



In [0]:
display(zone_names.take(5))

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone


## Task 1 - Filter rows

**Input:** trips dataset

### Task 1.1 - Remove "0 distance" and 'no passengers' records

Remove dataset rows that represent invalid trips:

- Trips where `trip_distance == 0` (no distance travelled)
- Trips where `passenger_count == 0` and `total_amount == 0` (we want to retain records where `total_amount` > 0 - these may be significant as the taxi may have carried some parcel, for example)

Altogether, a record is removed if it satisfies the following conditions:

`trip_distance == 0` or `(passenger_count == 0` and `total_amount == 0)`.

**Recommended:** Select only the relevant dataset columns for this and subsequent tasks: `['PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')]`

### Task 1.2 - Remove outliers using the modified z-score

Despite having removed spurious "zero passengers" trips in task 1.1, columns `total_amount` and `trip_distance` contain additional outlier values that must be identified and removed.

To identify and remove outliers, you will use the modified [z-score](https://en.wikipedia.org/wiki/Standard_score) method.
The modified z-score uses the median and [Median Absolute Deviation](https://en.wikipedia.org/wiki/Median_absolute_deviation) (MAD), instead of the mean and standard deviation, to determine how far an observation (indexed by i) is from the mean:

$$z_i = \frac{x_i - \mathit{median}(\mathbf{x})}{\mathbf{MAD}},$$

where x represents the input vector, xi is an element of x and zi is its corresponding z-score. In turn, the MAD formula is:

$$\mathbf{MAD} = 1.483 * \mathit{median}(\big\lvert x_i - \mathit{median}(\mathbf{x})\big\rvert).$$

Observations with **high** (absolute) z-score are considered outlier observations. A score is considered **high** if its __absolute z-score__ is larger than a threshold T = 3.5:

$$\big\lvert z_i \big\rvert > 3.5.$$

where T represents the number of unit standard deviations beyond which a score is considered an outlier ([wiki](https://en.wikipedia.org/wiki/68%E2%80%9395%E2%80%9399.7_rule)).

This process is repeated twice, once for each of the columns `total_amount` and `trip_distance` (in any order).

**Important:** Use the surrogate function [`percentile_approx`](https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.functions.percentile_approx.html?highlight=percentile#pyspark.sql.functions.percentile_approx) to estimate the median (calculating the median values for a column is expensive as it cannot be parallelised efficiently).

### Task 1.1

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
# Original Dataset
# display(trips)
display(trips.select('PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount'))
print_count(trips)

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3
114,90,1.6,1.0,12.8
90,144,1.8,1.0,13.3
114,48,2.0,1.0,14.75
48,152,5.7,1.0,22.3
234,148,1.8,1.0,14.75


Row count: 2,898,033


In [0]:
# Your solution implementation to task 1.1 goes HERE
def t11_remove_zeros(df):
    # input: trips dataset
    # Select only the relevant columns
    df = df.select('PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')
    # Remove trip_distance == 0
    df = df.filter(df.trip_distance != 0)
    # Remove passenger_count == 0 and total_amount == 0, but retain total_amount > 0 -> parcel
    df = df.filter(
        ((df.passenger_count != 0) & (df.total_amount > 0)) | 
        ((df.passenger_count == 0) & (df.total_amount > 0))
    )
    return df

In [0]:
# execute task 1.1
trips_11 = t11_remove_zeros(trips)

print_count(trips_11)

## uncomment only for smaller datasets
# display(trips_11.take(10))

Row count: 2,682,250


In [0]:
# After remove 0 and NO PASSENGER records
display(trips_11)
# display(trips_11.select('PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount'))
print_count(trips_11)

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3
114,90,1.6,1.0,12.8
90,144,1.8,1.0,13.3
114,48,2.0,1.0,14.75
48,152,5.7,1.0,22.3
234,148,1.8,1.0,14.75


Row count: 2,682,250


### Task 1.2

In [0]:
# Your solution implementation to task 1.2 goes HERE
def t12_remove_outliers(df):
    # Estimate Median of total_amount and trip_distance
    df_medians = df.select(
        pf.percentile_approx("total_amount", 0.5).alias('Median_total_amount'),
        pf.percentile_approx("trip_distance", 0.5).alias('Median_trip_distance')
    )
    df = df.join(df_medians)
    
    # Estimate MAD of total_amount and trip_distance
    ##Calculate 'absolute value of (x - Median)'
    df = df.withColumn(
        "Abs_Diff_total_amount", pf.abs(df.total_amount - df.Median_total_amount)
    ).withColumn(
        "Abs_Diff_trip_distance", pf.abs(df.trip_distance - df.Median_trip_distance)
    )
    ##Calculate 'Median(absolute(x - Median))'
    df_median_abs = df.select(
        pf.percentile_approx("Abs_Diff_total_amount", 0.5).alias('Median_Abs_total_amount'),
        pf.percentile_approx("Abs_Diff_trip_distance", 0.5).alias('Median_Abs_trip_distance')
    )
    df = df.join(df_median_abs)
    ##Calculate MAD
    df = df.withColumn(
        "MAD_total_amount", 1.483 * df.Median_Abs_total_amount
    ).withColumn(
        "MAD_trip_distance", 1.483 * df.Median_Abs_trip_distance
    )
    
    # Estimate absolute z-score of total_amount and trip_distance
    df = df.withColumn(
        "zScore_total_amount", pf.abs(
            (df.total_amount - df.Median_total_amount) / df.MAD_total_amount
        )
    ).withColumn(
        "zScore_trip_distance", pf.abs(
            (df.trip_distance - df.Median_trip_distance)/ df.MAD_trip_distance
        )
    )
    
    # Remove data with absolute z-score larger than 3.5
    df = df.filter(
        (df.zScore_total_amount <= 3.5) & 
        (df.zScore_trip_distance <= 3.5)
    )
    df = df.select('PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')
    
    return df

In [0]:
# execute task 1.2
trips_12 = t12_remove_outliers(trips_11)

print_count(trips_12)
# display(trips_12.take(10))

Row count: 2,371,848


In [0]:
display(trips_12)

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount
90,68,0.8,1.0,8.8
113,90,0.9,1.0,8.8
88,232,2.8,1.0,13.8
79,249,1.4,1.0,12.3
142,238,2.0,0.0,12.3
114,90,1.6,1.0,12.8
90,144,1.8,1.0,13.3
114,48,2.0,1.0,14.75
48,152,5.7,1.0,22.3
234,148,1.8,1.0,14.75


## Task 2 - Compute new columns

### Task 2.1 - Zone names

Obtain the **start** and **end** zone names of each trip by joining the `trips` and `zone_names` datasets (i.e. by using the `zone_names` dataset as lookup table).

**Note:** The columns containing the start and end zone ids of each trip are named `PULocationID` and `DOLocationID`, respectively.

### Task 2.2 - Unit profitability

Compute the column `unit_profitabilty = total_amount / trip_distance`.

### Task 2.1

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
display(zone_names)

LocationID,Borough,Zone,service_zone
1,EWR,Newark Airport,EWR
2,Queens,Jamaica Bay,Boro Zone
3,Bronx,Allerton/Pelham Gardens,Boro Zone
4,Manhattan,Alphabet City,Yellow Zone
5,Staten Island,Arden Heights,Boro Zone
6,Staten Island,Arrochar/Fort Wadsworth,Boro Zone
7,Queens,Astoria,Boro Zone
8,Queens,Astoria Park,Boro Zone
9,Queens,Auburndale,Boro Zone
10,Queens,Baisley Park,Boro Zone


In [0]:
# Your solution implementation to task 2.1 goes HERE
def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    # Add Start Zone Names
    df = df.join(zones_df.select('LocationID','Zone'), df.PULocationID == zones_df.LocationID)\
           .withColumnRenamed('Zone','start_zone').drop('LocationID')

    # Add End Zone Names
    df = df.join(zones_df.select('LocationID','Zone'), df.DOLocationID == zones_df.LocationID)\
           .withColumnRenamed('Zone','end_zone').drop('LocationID')

    return df

In [0]:
# execute task 2.1
trips_21 = t21_join_zones(trips_12, zones_df = zone_names)

print_count(trips_21)
# display(trips_21.take(10))

Row count: 2,371,848


In [0]:
display(trips_21)

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount,start_zone,end_zone
90,68,0.8,1.0,8.8,Flatiron,East Chelsea
113,90,0.9,1.0,8.8,Greenwich Village North,Flatiron
88,232,2.8,1.0,13.8,Financial District South,Two Bridges/Seward Park
79,249,1.4,1.0,12.3,East Village,West Village
142,238,2.0,0.0,12.3,Lincoln Square East,Upper West Side North
114,90,1.6,1.0,12.8,Greenwich Village South,Flatiron
90,144,1.8,1.0,13.3,Flatiron,Little Italy/NoLiTa
114,48,2.0,1.0,14.75,Greenwich Village South,Clinton East
48,152,5.7,1.0,22.3,Clinton East,Manhattanville
234,148,1.8,1.0,14.75,Union Sq,Lower East Side


### Task 2.2

In [0]:
# Your solution implementation to task 2.2 goes HERE
def t22_calc_profit(df):
    # input: output of task 2.1
    # Compute Unit Profitabilty
    df = df.withColumn("unit_profitability", df.total_amount / df.trip_distance)
    
    return df

In [0]:
# execute task 2.2
trips_22 = t22_calc_profit(trips_21)

print_count(trips_22)
# display(trips_22.take(10))

Row count: 2,371,848


In [0]:
display(trips_22)

PULocationID,DOLocationID,trip_distance,passenger_count,total_amount,start_zone,end_zone,unit_profitability
90,68,0.8,1.0,8.8,Flatiron,East Chelsea,11.0
113,90,0.9,1.0,8.8,Greenwich Village North,Flatiron,9.777777777777779
88,232,2.8,1.0,13.8,Financial District South,Two Bridges/Seward Park,4.928571428571429
79,249,1.4,1.0,12.3,East Village,West Village,8.785714285714286
142,238,2.0,0.0,12.3,Lincoln Square East,Upper West Side North,6.15
114,90,1.6,1.0,12.8,Greenwich Village South,Flatiron,8.0
90,144,1.8,1.0,13.3,Flatiron,Little Italy/NoLiTa,7.388888888888889
114,48,2.0,1.0,14.75,Greenwich Village South,Clinton East,7.375
48,152,5.7,1.0,22.3,Clinton East,Manhattanville,3.912280701754386
234,148,1.8,1.0,14.75,Union Sq,Lower East Side,8.194444444444445


## Task 3: Rank zones by traffic, passenger volume and profitability

### 3.1 - Summarise interzonal travel

Build a graph data structure of zone-to-zone traffic, representing aggregated data about trips between any two zones. The graph will have one node for each zone and one edge connecting each pair of zones. In addition, edges contain aggregate information about all trips between those zones. 

For example, zones Z1 and Z2 are connected by *two* edges: edge Z1 --> Z2 carries aggregate data about all trips that originated in Z1 and ended in Z2, and edge Z2 --> Z2 carries aggregate data about all trips that originated in Z2 and ended in Z1.

The aggregate information of interzonal travel must include the following data:

- `average_unit_profit` - the average unit profitability (calculated as `mean(unit_profitabilty)`).
- `trips_count` -- the total number of recorded trips.
- `total_passengers` -- the total number of passenger across all trips (sum of `passenger_count`).

This graph can be represented as a new dataframe, with schema:

\[`PULocationID`, `DOLocationID`, `average_unit_profit`, `trips_count`, `total_passengers` \]

__hint__: the `groupby()` operator produces a `pyspark.sql.GroupedData` structure. You can then calculate multiple aggregations from this using `pyspark.sql.GroupedData.agg()`: 
- https://spark.apache.org/docs/3.2.0/api/python/reference/pyspark.pandas/api/pyspark.pandas.DataFrame.groupby.html
- https://spark.apache.org/docs/3.2.0/api/python/reference/api/pyspark.sql.GroupedData.agg.html

### Task 3.2 - Obtain top-10 zones

For each of the following measures, report the top-10 zones _using their plain names you dereferenced in the previous step, not the codes_. Note that this requires ranking the nodes in different orders. Specifically, you need to calculate the following further aggregations:

- the **total** number of trips originating from Z. This is simply the sum of `trips_count` over all outgoing edges for Z, i.e., edges of the form Z -> \*
- the **average** profitability of a zone. This is the average of all `average_unit_profit` over all *outgoing* edges from Z.
- The **total** passenger volume measured as the **sum** of `total_passengers` carried in trips that originate from Z

### Task 3.1

In [0]:
# develop your solution here (create/destroy cells as needed) and then implement it in the functions below

In [0]:
## Your solution to task 3.1 goes HERE
def t31_summarise_trips(df):
    # input: output of task 2.2
    # Compute average_unit_profit, trips_count and total_passengers
    df = df.groupby("PULocationID", "DOLocationID").agg(
        pf.mean("unit_profitability").alias('average_unit_profit'), 
        pf.count("trip_distance").alias('trips_count'), 
        pf.sum("passenger_count").alias('total_passengers')
    )
    
    return df

In [0]:
# execute task 3.1
graph = t31_summarise_trips(trips_22)

print_count(graph)
# display(graph.take(10))

Row count: 10,022


In [0]:
display(graph)

PULocationID,DOLocationID,average_unit_profit,trips_count,total_passengers
90,231,7.396050675624754,1094,1576.0
87,33,6.817160729833857,105,145.0
90,142,6.857922958972675,883,1276.0
114,100,7.451603625245185,452,705.0
234,144,9.112465640948626,1465,2186.0
148,262,4.800669891200868,248,398.0
246,249,8.760901617465663,1108,1713.0
170,179,5.109737860724155,76,101.0
48,232,5.427765741716741,317,496.0
142,144,5.754774114280266,326,485.0


### Task 3.2

In [0]:
# Your solution to task 3.2 goes HERE (implement each of the functions below)
def t32_summarise_zones_pairs(df, zones_df = zone_names):
    # Repeat task 2.1 to recover zone names
    ##Add Start Zone Names
    df = df.join(zones_df.select('LocationID','Zone'), df.PULocationID == zones_df.LocationID)\
           .withColumnRenamed('Zone','start_zone').drop('LocationID')
    ##Add End Zone Names
    df = df.join(zones_df.select('LocationID','Zone'), df.DOLocationID == zones_df.LocationID)\
           .withColumnRenamed('Zone','end_zone').drop('LocationID')
    
    return df

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    # Compute the Total number of trips
    df_zones = df_zones.groupby("start_zone")\
                       .agg(pf.sum("trips_count").alias('Total_trips'))\
                       .sort("Total_trips", ascending = False)\
                       .limit(10)\
                       .toPandas()
    
    return df_zones

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    # Compute the Average profitability of zone
    df_zones = df_zones.groupby("start_zone")\
                       .agg(pf.avg("average_unit_profit").alias('Average_profitability'))\
                       .sort("Average_profitability", ascending = False)\
                       .limit(10)\
                       .toPandas()
    
    return df_zones

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
    # Compute the Total passenger volume measured as the sum
    df_zones = df_zones.groupby("start_zone")\
                       .agg(pf.sum("total_passengers").alias('Total_passengers'))\
                       .sort("Total_passengers", ascending = False)\
                       .limit(10)\
                       .toPandas()
    
    return df_zones

In [0]:
# execute task 3.2
zones = t32_summarise_zones_pairs(graph)

top10_trips     = t32_top10_trips(zones)
top10_profit    = t32_top10_profit(zones)
top10_passenger = t32_top10_passenger(zones)

In [0]:
display(zones)
print_count(zones)

PULocationID,DOLocationID,average_unit_profit,trips_count,total_passengers,start_zone,end_zone
90,231,7.396050675624754,1094,1576.0,Flatiron,TriBeCa/Civic Center
87,33,6.817160729833857,105,145.0,Financial District North,Brooklyn Heights
90,142,6.857922958972675,883,1276.0,Flatiron,Lincoln Square East
114,100,7.451603625245185,452,705.0,Greenwich Village South,Garment District
234,144,9.112465640948626,1465,2186.0,Union Sq,Little Italy/NoLiTa
148,262,4.800669891200868,248,398.0,Lower East Side,Yorkville East
246,249,8.760901617465663,1108,1713.0,West Chelsea/Hudson Yards,West Village
170,179,5.109737860724155,76,101.0,Murray Hill,Old Astoria
48,232,5.427765741716741,317,496.0,Clinton East,Two Bridges/Seward Park
142,144,5.754774114280266,326,485.0,Lincoln Square East,Little Italy/NoLiTa


Row count: 10,022


In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
top10_trips

,start_zone,Total_trips
0,Upper East Side South,119874
1,Upper East Side North,101505
2,Penn Station/Madison Sq West,95770
3,Midtown Center,92093
4,Murray Hill,86251
5,Midtown East,83735
6,Lincoln Square East,79180
7,Lenox Hill West,72533
8,Clinton East,72509
9,Upper West Side South,72484


In [0]:
# use 'display()' return a pandas DataFrame for 'pretty' output
top10_profit

,start_zone,Average_profitability
0,Queensboro Hill,259.104384
1,Baisley Park,193.068838
2,Prospect Park,169.175210
3,Eastchester,107.234348
4,Saint Michaels Cemetery/Woodside,73.315362
5,Van Cortlandt Village,67.233919
6,Crotona Park,62.131449
7,Riverdale/North Riverdale/Fieldston,62.018521
8,Flatbush/Ditmas Park,58.744308
9,South Jamaica,48.728374


In [0]:
# use 'display()' or return a pandas DataFrame for 'pretty' output
top10_passenger

,start_zone,Total_passengers
0,Upper East Side South,170039.0
1,Upper East Side North,144032.0
2,Penn Station/Madison Sq West,137159.0
3,Midtown Center,134853.0
4,Murray Hill,123799.0
5,Midtown East,119723.0
6,Lincoln Square East,114752.0
7,Clinton East,107120.0
8,Upper West Side South,105888.0
9,East Village,105571.0


## Task 4 - Record the pipeline's execution time

Record the execution time of:

1. the whole pipeline
2. the whole pipeline except task 1.2

on the two tables below, for all dataset sizes: `'S'`, `'M'`, `'L'`, `'XL'`, `'XXL'`, and data formats: `parquet` and `delta`.

Analyse the resulting execution times and comment on the effect of dataset size, dataset format and task complexity (with and without task 1.2) on pipeline performance.

### Initialise

In [0]:
# after developing your solution, it may be convenient to combine all your functions in a single cell (at the start or end of the notebook)

# CHANGE the value of the following arguments to record the pipeline execution times for increasing dataset sizes
SIZE = 'L'   # S, M, L, XL, XXL
DATA_FORMAT = 'parquet'   # parquet, delta
WITH_TASK_12 = True    # True, False

# Load trips dataset
trips = init_trips(SIZE, DATA_FORMAT)


    Trips dataset loaded!
    ---
      Size: L
      Format: parquet
      Tables loaded: ['/FileStore/tables/taxi/parquet/tripdata_2020_21.parquet']
      Number of trips (dataset rows): 41,953,716
    


### All Solutions

In [0]:
# ALL DEVELOPED SOLUTIONS #

# Task 1.1
def t11_remove_zeros(df):
    # input: trips dataset
    # Select only the relevant columns
    df = df.select('PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')
    # Remove trip_distance == 0
    df = df.filter(df.trip_distance != 0)
    # Remove passenger_count == 0 and total_amount <= 0, but retain total_amount > 0 -> parcel
    df = df.filter(
        ((df.passenger_count != 0) & (df.total_amount > 0)) | 
        ((df.passenger_count == 0) & (df.total_amount > 0))
    )
    return df

# Task 1.2
def t12_remove_outliers(df):
    # Estimate Median of total_amount and trip_distance
    df_medians = df.select(
        pf.percentile_approx("total_amount", 0.5).alias('Median_total_amount'),
        pf.percentile_approx("trip_distance", 0.5).alias('Median_trip_distance')
    )
    df = df.join(df_medians)
    
    # Estimate MAD of total_amount and trip_distance
    ##Calculate 'absolute value of (x - Median)'
    df = df.withColumn(
        "Abs_Diff_total_amount", pf.abs(df.total_amount - df.Median_total_amount)
    ).withColumn(
        "Abs_Diff_trip_distance", pf.abs(df.trip_distance - df.Median_trip_distance)
    )
    ##Calculate 'Median(absolute(x - Median))'
    df_median_abs = df.select(
        pf.percentile_approx("Abs_Diff_total_amount", 0.5).alias('Median_Abs_total_amount'),
        pf.percentile_approx("Abs_Diff_trip_distance", 0.5).alias('Median_Abs_trip_distance')
    )
    df = df.join(df_median_abs)
    ##Calculate MAD
    df = df.withColumn(
        "MAD_total_amount", 1.483 * df.Median_Abs_total_amount
    ).withColumn(
        "MAD_trip_distance", 1.483 * df.Median_Abs_trip_distance
    )
    
    # Estimate absolute z-score of total_amount and trip_distance
    df = df.withColumn(
        "zScore_total_amount", pf.abs(
            (df.total_amount - df.Median_total_amount) / df.MAD_total_amount
        )
    ).withColumn(
        "zScore_trip_distance", pf.abs(
            (df.trip_distance - df.Median_trip_distance)/ df.MAD_trip_distance
        )
    )
    
    # Remove data with absolute z-score larger than 3.5
    df = df.filter(
        (df.zScore_total_amount <= 3.5) & 
        (df.zScore_trip_distance <= 3.5)
    )
    df = df.select('PULocationID', 'DOLocationID', 'trip_distance', 'passenger_count', 'total_amount')
    
    return df

# Task 2.1
def t21_join_zones(df, zones_df = zone_names):
    # input: output of task 1.2 and zone_names dataset
    # Add Start Zone Names
    df = df.join(zones_df.select('LocationID','Zone'), df.PULocationID == zones_df.LocationID)\
           .withColumnRenamed('Zone','start_zone').drop('LocationID')

    # Add End Zone Names
    df = df.join(zones_df.select('LocationID','Zone'), df.DOLocationID == zones_df.LocationID)\
           .withColumnRenamed('Zone','end_zone').drop('LocationID')

    return df

# Task 2.2
def t22_calc_profit(df):
    # input: output of task 2.1
    # Compute Unit Profitabilty
    df = df.withColumn("unit_profitability", df.total_amount / df.trip_distance)
    
    return df

# Task 3.1
def t31_summarise_trips(df):
    # input: output of task 2.2
    # Compute average_unit_profit, trips_count and total_passengers
    df = df.groupby("PULocationID", "DOLocationID").agg(
        pf.mean("unit_profitability").alias('average_unit_profit'), 
        pf.count("trip_distance").alias('trips_count'), 
        pf.sum("passenger_count").alias('total_passengers')
    )
    
    return df

# Task 3.2
def t32_summarise_zones_pairs(df, zones_df = zone_names):
    # Repeat task 2.1 to recover zone names
    ##Add Start Zone Names
    df = df.join(zones_df.select('LocationID','Zone'), df.PULocationID == zones_df.LocationID)\
           .withColumnRenamed('Zone','start_zone').drop('LocationID')
    ##Add End Zone Names
    df = df.join(zones_df.select('LocationID','Zone'), df.DOLocationID == zones_df.LocationID)\
           .withColumnRenamed('Zone','end_zone').drop('LocationID')
    
    return df

# Top 10 ranked zones by traffic (trip volume)
def t32_top10_trips(df_zones):
    # input: output of task 3.2
    # Compute the Total number of trips
    df_zones = df_zones.groupby("start_zone")\
                       .agg(pf.sum("trips_count").alias('Total_trips'))\
                       .sort("Total_trips", ascending = False)\
                       .limit(10)\
                       .toPandas()
    
    return df_zones

# Top 10 ranked zones by profit
def t32_top10_profit(df_zones):
    # input: output of task 3.2
    # Compute the Average profitability of zone
    df_zones = df_zones.groupby("start_zone")\
                       .agg(pf.avg("average_unit_profit").alias('Average_profitability'))\
                       .sort("Average_profitability", ascending = False)\
                       .limit(10)\
                       .toPandas()
    
    return df_zones

# Top 10 ranked zones by passenger volume
def t32_top10_passenger(df_zones):
    # input: output of task 3.2
    # Compute the Total passenger volume measured as the sum
    df_zones = df_zones.groupby("start_zone")\
                       .agg(pf.sum("total_passengers").alias('Total_passengers'))\
                       .sort("Total_passengers", ascending = False)\
                       .limit(10)\
                       .toPandas()
    
    return df_zones

### Run & Record

In [0]:
# run and record the resulting execution time shown by databricks (on the cell footer)

# IMPORTANT: this function calls all task functions in order of occurrence. For this code to run without errors, you have to load into memory all of the previous task-specific functions, even if you haven't implemented these yet.
pipeline(trips, with_task_12 = WITH_TASK_12)

Out[81]: [                     start_zone  Total_trips
 0         Upper East Side South      1819890
 1         Upper East Side North      1715429
 2                Midtown Center      1334351
 3  Penn Station/Madison Sq West      1250953
 4                  Midtown East      1179117
 5           Lincoln Square East      1142439
 6                   Murray Hill      1121997
 7         Upper West Side South      1099328
 8               Lenox Hill West      1074113
 9                  Clinton East      1013937,
                          start_zone  Average_profitability
 0                    Newark Airport              91.735409
 1                  Sunset Park East              90.008716
 2                          Canarsie              76.334374
 3                         Glen Oaks              72.828478
 4                Murray Hill-Queens              62.618637
 5                          Rosedale              56.228450
 6                       JFK Airport              53.389259
 7  

_Table 1. Pipeline performance for `parquet` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    | 2.9  | 15.6 |  42.0 |  90.4 |  132.4 |
| execution time   (w/o 1.2)  | 7.00sec | 1.61min | 13.54min | 19.16min | 28.92min |
| execution time              | 23.02sec | 4.59min | 36.94min | 50.91min | 35.04min |
| sec / 1M records (w/o 1.2)  | 2.41  | 6.19  | 19.34  | 12.72  | 13.11  |
| sec / 1M records            | 7.94  | 17.65  | 52.77  | 33.79 | 15.88  |

_Table 2. Pipeline performance for `delta` format._

| metric                      | S    | M    | L    | XL   | XXL  |
|-----------------------------|------|------|------|------|------|
| rows (M)                    |  2.9 |  15.6 |  42.0 |  90.4 |  132.4 |
| execution time   (w/o 1.2)  | 7.24sec | 10.45sec | 9.54sec | 10.61sec | 10.33sec |
| execution time              | 12.60sec | 29.44sec | 31.18sec | 49.52sec | 1.07min |
| sec / 1M records (w/o 1.2)  | 2.50  | 0.67  | 0.23  | 0.12  | 0.08  |
| sec / 1M records            | 4.34  | 1.89  | 0.74  | 0.55  | 0.48  |